<a href="https://colab.research.google.com/github/jing-zhou/w2v/blob/master/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Environment setup and library import**

In [3]:
import random                           # Generates random numbers
import os                               # Create directories, list files
from zipfile import ZipFile
from shutil import copyfile             # Copy files from Source to Destination
import cv2                              # To flip images in data augmentation
import matplotlib.pyplot as plt         # To save the images
import matplotlib.image as mpimg

**2. Link Goggle Drive with Colab**

Source : https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d 

In [ ]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

**3. handler function for zip file**

In [5]:
# function for handling zip file, zfile must be FQN
def handleZip(corp, zfile, temp):
  # Create a ZipFile Object and load sample.zip in it
  with ZipFile(zfile, 'r') as zipObj:
    # Extract all the contents of zip file in different directory
    zipObj.extractall(temp)
    # Get a list of all archived file names from the zip
    listOfFileNames = zipObj.namelist()
    # Iterate over the file names
    for fileName in listOfFileNames:
      if fileName.endswith('.zip'):
        handleZip(corp, temp + '/' + fileName, temp)
        continue

      tfile = open(temp + '/' + fileName, 'r')
      corp.write(tfile.read())
      tfile.close()


**4. Create root dataset directory, temp directory and corpus file**


In [58]:
dataset_dir = 'dataset'
if not os.path.exists(dataset_dir):
  os.mkdir(dataset_dir)

corpus_dir = dataset_dir + '/corpus'
if not os.path.exists(corpus_dir):
  os.mkdir(corpus_dir)
corpus_file = '/corpus.txt'
# remove previous file instance
if os.path.exists(corpus_dir + corpus_file):
  os.remove(corpus_dir + corpus_file)

corpus = open(corpus_dir + corpus_file, 'x')
# corpus.write("Now the file has more content!")
# corpus.close()
# !cat /content/dataset/corpus.txt

temp_dir = 'temp'
if not os.path.exists(temp_dir):
  os.mkdir(temp_dir)

gdrive = "GDrive/My Drive/corpus/segmented/"

file_list = os.listdir (gdrive)
# print(file_list)

for fname in file_list:
  if fname.endswith('.zip'):
    handleZip(corpus, gdrive + fname, temp_dir)
    continue

  tmp = open(gdrive + fname, 'r')
  corpus.write(tmp.read())
  tmp.close()

corpus.close()
# !cat /content/dataset/corpus.txt


**5. W2V trained by Gensim**

In [ ]:
from gensim.models.word2vec import PathLineSentences
import multiprocessing
import gensim.models

model = gensim.models.Word2Vec(
    sentences=PathLineSentences(corpus_dir + corpus_file),
    min_count=1,
    size=300,
    window=15,
    workers=multiprocessing.cpu_count(),
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
    iter=5
    )

**6. display the W2V**

In [ ]:
for i, word in enumerate(model.wv.vocab):
  if i == 10:
    break
  print(word)
  print(model.wv[word])

**7. Store the model file**

In [ ]:
models_dir = dataset_dir + '/models'
if not os.path.exists(models_dir):
  os.mkdir(models_dir)
model_file = '/model.txt'
# remove previous file instance
if os.path.exists(models_dir + model_file):
  os.remove(models_dir + model_file)

model.save(models_dir + model_file)

**8. Demo**

In [ ]:
print('%r\t%r\t%.2f' % ("维蒂格豪森", "迪欣根", model.similarity("维蒂格豪森", "迪欣根")))